<a href="https://colab.research.google.com/github/quocthang0507/tensorflow_text/blob/main/text_classification_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Copyright 2018 The TensorFlow Hub Authors.

```
@title Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License.
You may obtain a copy of the License at
https://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
```

# Phân loại văn bản với RNN

Hướng dẫn phân loại văn bản này huấn luyện một [mạng thần kinh hồi quy](https://developers.google.com/machine-learning/glossary/#recurrent_neural_network) trên [tập dữ liệu lớn đánh giá phim IMDB](http://ai.stanford.edu/~amaas/data/sentiment/) để phân tích cảm xúc.

## Cài đặt

In [ ]:
import numpy as np

import tensorflow_datasets as tfds
import tensorflow as tf

tfds.disable_progress_bar()

Nhập `matplotlib` và tạo một hàm trợ giúp để vẽ đồ thị:



In [ ]:
import matplotlib.pyplot as plt


def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])

## Thiết lập đường tin đầu vào

Tập dữ liệu lớn đánh giá phim IMDB là tập dữ liệu *phân loại nhị phân* — tất cả các bài đánh giá đều có ý nghĩa *tích cực* hoặc *tiêu cực*.

Tải về tập dữ liệu bằng [TFDS](https://www.tensorflow.org/datasets). Xem [hướng dẫn tải văn bản](https://www.tensorflow.org/tutorials/load_data/text) để biết chi tiết về cách tải dữ liệu này theo cách thủ công.

In [ ]:
dataset, info = tfds.load('imdb_reviews', with_info=True,
                          as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

train_dataset.element_spec

Ban đầu, điều này trả về một tập dữ liệu của `(text, label pairs)`:

In [ ]:
for example, label in train_dataset.take(1):
  print('text: ', example.numpy())
  print('label: ', label.numpy())

Tiếp theo xáo trộn dữ liệu huấn luyện và tạo các lô của các cặp `(text, label)`:

In [ ]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [ ]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
for example, label in train_dataset.take(1):
  print('texts: ', example.numpy()[:3])
  print()
  print('labels: ', label.numpy()[:3])

## Tạo bộ mã hóa văn bản

Văn bản thô được tải bởi `tfds` cần phải được xử lý trước khi nó có thể được sử dụng trong một mô hình. Cách đơn giản nhất để xử lý văn bản cho huấn luyện là sử dụng lớp `experimental.preprocessing.TextVectorization`. Lớp này có nhiều khả năng, nhưng hướng dẫn này chọn hành vi mặc định.

Tạo lớp và truyền văn bản của tập dữ liệu vào phương thức `.adapt` của lớp:

In [ ]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

Phương thức `.adapt` đặt từ vựng của lớp. Đây là 20 token đầu tiên. Sau phần đệm và các token không xác định, chúng được sắp xếp theo tần suất:

In [ ]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

Sau khi từ vựng được thiết lập, lớp có thể mã hóa văn bản thành các chỉ mục. Các tensor của chỉ số được đệm-0 vào chuỗi dài nhất trong lô (trừ khi bạn đặt `output_sequence_length` cố định):

In [ ]:
encoded_example = encoder(example)[:3].numpy()
encoded_example

Với cài đặt mặc định, quá trình này không thể hoàn toàn đảo ngược. Có ba lý do chính cho điều đó:

1. Giá trị mặc định cho `preprocessing.TextVectorization`'s đối số `standardize` là `"lower_and_strip_punctuation"`.

2. Kích thước từ vựng hạn chế và thiếu dự phòng dựa trên ký tự dẫn đến một số token không xác định.

In [ ]:
for n in range(3):
  print("Original: ", example[n].numpy())
  print("Round-trip: ", " ".join(vocab[encoded_example[n]]))
  print()

## Tạo mô hình

![bidirectional](img/bidirectional.png)

Trên đây là sơ đồ của mô hình.

1. Mô hình này có thể được xây dựng dưới dạng `tf.keras.Sequential`.

2. Lớp đầu tiên là `encoder` chuyển đổi văn bản thành một chuỗi các chỉ số token.

3. Sau bộ mã hóa là một lớp nhúng. Một lớp nhúng lưu trữ một vectơ trên mỗi từ. Khi được gọi, nó sẽ chuyển chuỗi các chỉ số từ thành chuỗi các vectơ. Các vectơ này có thể huấn luyện được. Sau khi huấn luyện (trên đủ dữ liệu), các từ có nghĩa tương tự thường có các vectơ tương tự.

 Tra cứu chỉ mục này hiệu quả hơn nhiều so với hoạt động tương đương của việc truyền một vectơ được mã hóa thông qua một lớp `tf.keras.layers.Dense`.

4. Mạng thần kinh đệ quy (RNN) xử lý chuỗi đầu vào bằng cách lặp đi lặp lại các phần tử. Các RNN chuyển các kết quả đầu ra từ bước này sang đầu vào của chúng ở bước thời gian tiếp theo.

 Trình bao bọc `tf.keras.layers.Bidirectional` cũng có thể được sử dụng với một lớp RNN. Điều này truyền đầu vào chuyển tiếp và ngược qua lớp RNN và sau đó nối đầu ra cuối cùng.

 * Ưu điểm chính của RNN hai chiều là tín hiệu từ bắt đầu của đầu vào không cần phải được xử lý toàn bộ qua mỗi bước thời gian để ảnh hưởng đến đầu ra.

 * Nhược điểm chính của RNN hai chiều là bạn không thể truyền các dự đoán một cách hiệu quả vì các từ đang được thêm vào cuối.

5. Sau khi RNN đã chuyển đổi chuỗi thành một vectơ duy nhất, hai `layers.Dense` sẽ thực hiện một số xử lý cuối cùng và chuyển đổi từ biểu diễn vectơ này thành một logit duy nhất làm đầu ra phân loại.

Mã để thực hiện điều này là bên dưới:

In [ ]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

Xin lưu ý rằng mô hình tuần tự Keras được sử dụng ở đây vì tất cả các lớp trong mô hình chỉ có đầu vào duy nhất và tạo ra đầu ra duy nhất. Trong trường hợp bạn muốn sử dụng lớp RNN trạng thái, bạn có thể muốn xây dựng mô hình của mình với API chức năng Keras hoặc lớp con mô hình để bạn có thể truy xuất và sử dụng lại các trạng thái của lớp RNN. Vui lòng kiểm tra [hướng dẫn Keras RNN](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse) để biết thêm chi tiết.

Lớp nhúng [sử dụng mặt nạ](https://www.tensorflow.org/guide/keras/masking_and_padding) để xử lý các độ dài chuỗi khác nhau. Tất cả các lớp sau khi tạo mặt nạ hỗ trợ `Embedding`:

In [ ]:
print([layer.supports_masking for layer in model.layers])

Để xác nhận rằng điều này hoạt động như mong đợi, hãy đánh giá một câu hai lần. Đầu tiên, một mình vì vậy không có phần đệm để che giấu:

In [ ]:
# predict on a sample text without padding.

sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')
predictions = model.predict(np.array([sample_text]))
print(predictions[0])

Bây giờ, hãy đánh giá lại nó trong một lô với một câu dài hơn. Kết quả phải giống hệt nhau:

In [ ]:
# predict on a sample text with padding

padding = "the " * 2000
predictions = model.predict(np.array([sample_text, padding]))
print(predictions[0])

Biên dịch mô hình Keras để định cấu hình quá trình huấn luyện:

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

## Huấn luyện mô hình

In [ ]:
history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset,
                    validation_steps=30)

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

In [ ]:
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plot_graphs(history, 'accuracy')
plt.ylim(None, 1)
plt.subplot(1, 2, 2)
plot_graphs(history, 'loss')
plt.ylim(0, None)

Chạy dự đoán cho một câu mới:

Nếu dự đoán là `>= 0.0`, nó là tích cực, còn nó là tiêu cực.

In [ ]:
sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')
predictions = model.predict(np.array([sample_text]))

## Xếp chồng hai hoặc nhiều lớp LSTM

Các lớp đệ quy của Keras có hai chế độ khả dụng được điều khiển bởi đối số phương thức `return_sequences`:

Nếu `False` nó chỉ trả về đầu ra cuối cùng cho mỗi chuỗi đầu vào (hình dạng tensor 2D `(batch_size, output_features)`). Đây là mặc định được sử dụng trong mô hình trước đó.

Nếu `True`, chuỗi đầy đủ của các đầu ra liên tiếp cho mỗi bước thời gian sẽ được trả về (một hình dạng 3D `(batch_size, timesteps, output_features)`).

Đây là luồng thông tin trông như thế nào với `return_sequences=True`:

![layered_bidirectional](img/layered_bidirectional.png)

Điều thú vị khi sử dụng RNN với `return_sequences=True` là đầu ra vẫn có 3 trục, giống như đầu vào, vì vậy nó có thể được chuyển sang một lớp RNN khác, như thế này:

In [ ]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(len(encoder.get_vocabulary()), 64, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset,
                    validation_steps=30)

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

In [ ]:
# predict on a sample text without padding.

sample_text = ('The movie was not good. The animation and the graphics '
               'were terrible. I would not recommend this movie.')
predictions = model.predict(np.array([sample_text]))
print(predictions)

In [ ]:
plt.figure(figsize=(16, 6))
plt.subplot(1, 2, 1)
plot_graphs(history, 'accuracy')
plt.subplot(1, 2, 2)
plot_graphs(history, 'loss')

Kiểm tra các lớp lặp lại hiện có khác chẳng hạn như [các lớp GRU](https://www.tensorflow.org/api_docs/python/tf/keras/layers/GRU).

Nếu bạn đang quan tâm đến việc xây dựng các RNN tùy chỉnh, hãy xem [Hướng dẫn Keras RNN](https://www.tensorflow.org/guide/keras/rnn).